# Database Connection

## Import libraries

In [4]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import pymysql

## MySql Terminal and Password

In [7]:
host_name = "localhost"
user_name = "root"
pass_word = 'Hari@123'
database_name = 'student_information_system'

In [9]:
def create_server_connection (host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host_name,
            user = user_name,
            password = user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

# Connect to database
connection = create_server_connection(host_name, user_name, pass_word)

MySQL Database connection successful


## Connect to Database

In [12]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host = host_name,
            user = user_name,
            password = user_password,
            database = db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

## Execute Sql queries

In [15]:
def execute_query(connection, query, data=None):
    cursor = connection.cursor()
    try:
        if data:
            cursor.execute(query, data)
        else:
            cursor.execute(query)
        connection.commit()
        # print("Query was successful")
        return True
    except Error as err:
        print(f"Error : '{err}' ")
        return False

## query for create order table

## Connect to the database

In [19]:
connection = create_db_connection(host_name, user_name, pass_word, database_name)

MySQL Database connection successful


In [21]:
connection

## Insert Data in Database

## Read the data from table

In [25]:
def read_query(connection, query, data=None):
    cursor = connection.cursor()
    result = None
    try:
        if data:
            cursor.execute(query, data)
        else:
            cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error : '{err}' ")
    finally:
        cursor.close()

## Get all data from database

In [28]:
get_query = """
Select * from Users;
"""

## Store the output in variable as output etc..

In [31]:
output = read_query(connection, get_query)

## Print the row using for loop

## Convert data into Table using Pandas

In [35]:
from_db = []

for row in output:
    result = list(row)
    from_db.append(result)

## Table columns

In [38]:
User_column = ['user_id', 'username', 'password', 'user_role']

In [40]:
User_table = pd.DataFrame(from_db, columns = User_column)

In [42]:
User_table

,user_id,username,password,user_role
0,1,john_doe,password123,staff
1,2,alice_smith,alicepwd456,student
2,3,michael_lee,mikepass789,staff
3,4,emma_jones,emma@2023,student
4,5,david_brown,davepwd101,staff
5,6,susan_clark,susie@789,student
6,7,tom_harris,tomh1234,staff
7,8,lucy_king,lucy@2024,student
8,9,george_wilson,georgepwd999,staff
9,10,isla_davis,isla2023!,student


# Student_information_Sytem

In [45]:
import tkinter as tk
from tkinter import ttk, messagebox

## Login functionality

In [48]:
def login():
    # Retrieve entered username and password
    username = username_entry.get()
    password = password_entry.get()

    # Simple SQL query to check if the username and password match
    query_check = """
    SELECT COUNT(*) FROM Users WHERE username = %s AND password = %s;
    """

    # Connect to the database and validate credentials
    try:
        connection = create_db_connection(host_name, user_name, pass_word, database_name)
        result = read_query(connection, query_check, (username, password))
        # print(result)
        
        # If result[0] > 0, login is successful
        if result and result[0][0] > 0:
            messagebox.showinfo("Login Success", "Welcome to the Student Information System!")
            login_window.destroy()  # Close the login window
            main_menu()  # Open the main application window
        else:
            messagebox.showerror("Login Failed", "Invalid username or password!")
    except pymysql.MySQLError as e:
        messagebox.showerror("Database Error", f"Error connecting to database: {e}")
    finally:
        if connection:
            connection.close()  # Make sure to close the connection

## Operation: Add a New Student

In [51]:
def add_new_student():
    window = tk.Toplevel()
    window.title("Add New Student")
    
    fields = ["Name", "DOB", "Gender", "Address", "Phone", "Email"]
    entries = {}
    for idx, field in enumerate(fields):
        tk.Label(window, text=field).grid(row=idx, column=0, padx=10, pady=5)
        entry = tk.Entry(window)
        entry.grid(row=idx, column=1, padx=10, pady=5)
        entries[field] = entry

    def submit():
        connection = create_db_connection(host_name, user_name, pass_word, database_name)
        if connection:
            query = "INSERT INTO Student (name, dob, gender, address, phone, email) VALUES (%s, %s, %s, %s, %s, %s)"
            values = tuple(entries[field].get() for field in fields)
            success = execute_query(connection, query, values)
            if success:
                messagebox.showinfo("Success", "Student added successfully!")
            else:
                messagebox.showerror("Error", "Failed to add student.")
            connection.close()
        else:
            messagebox.showerror("Error", "Failed to connect to the database.")

    tk.Button(window, text="Submit", command=submit).grid(row=len(fields), column=0, columnspan=2, pady=10)

## Operation: Update Student Details

In [54]:
# Operation: Update Student Details
def update_student_details():
    window = tk.Toplevel()
    window.title("Update Student Details")
    
    # List of fields to be updated
    fields = ["Student ID", "Name", "DOB", "Gender", "Address", "Phone", "Email"]
    entries = {}

    # Create input fields for each of the fields
    for idx, field in enumerate(fields):
        tk.Label(window, text=field).grid(row=idx, column=0, padx=10, pady=5)
        entry = tk.Entry(window)
        entry.grid(row=idx, column=1, padx=10, pady=5)
        entries[field] = entry

    # Submit function to handle the update query
    def submit():
        student_id = entries["Student ID"].get()  # Get the student ID from the entry field
        if not student_id:
            messagebox.showwarning("Input Error", "Student ID is required!")
            return
        
        # Gather the values for the fields
        values = tuple(entries[field].get() for field in fields[1:])  # Skip Student ID field
        if not any(values):  # If no fields are filled for updating
            messagebox.showwarning("Input Error", "No fields to update!")
            return
        
        # Create database connection
        connection = create_db_connection(host_name, user_name, pass_word, database_name)
        
        if connection:
            # Create dynamic update query
            updates = []
            update_values = []
            
            # Construct the update query dynamically
            for field, entry in entries.items():
                if field != "Student ID" and entry.get():  # Skip "Student ID"
                    db_field = field.lower().replace(" ", "_")  # Column name should match DB field
                    updates.append(f"{db_field}=%s")
                    update_values.append(entry.get())
            
            # Add the student_id as the last value for the WHERE clause
            query = f"UPDATE Student SET {', '.join(updates)} WHERE student_id=%s"
            values_for_query = update_values + [student_id]
            
            # Execute the query and commit changes
            success = execute_query(connection, query, values_for_query)
            
            if success:
                messagebox.showinfo("Success", "Student details updated successfully!")
                window.destroy()  # Close the window
            else:
                messagebox.showerror("Error", "Failed to update student details.")
            
            connection.close()
        else:
            messagebox.showerror("Error", "Failed to connect to the database.")

    # Submit button
    tk.Button(window, text="Submit", command=submit).grid(row=len(fields), column=0, columnspan=2, pady=10)

## Operation: Remove Student

In [57]:
# Operation: Remove Student
def remove_student():
    operation_window = tk.Toplevel()
    operation_window.title("Remove Student")

    # Create Student ID input field
    tk.Label(operation_window, text="Student ID").grid(row=0, column=0, padx=10, pady=5)
    student_id_entry = tk.Entry(operation_window)
    student_id_entry.grid(row=0, column=1, padx=10, pady=5)

    # Function to handle deletion of student
    def delete():
        student_id = student_id_entry.get()
        if not student_id:
            messagebox.showwarning("Input Error", "Student ID is required!")
            return
        
        # Create database connection
        connection = create_db_connection(host_name, user_name, pass_word, database_name)
        
        if connection:
            # SQL DELETE query
            query = "DELETE FROM Student WHERE student_id=%s"
            values = (student_id,)  # Use tuple to pass parameters for query execution
            
            # Execute the query to remove the student
            success = execute_query(connection, query, values)
            
            if success:
                messagebox.showinfo("Success", "Student removed successfully!")
                operation_window.destroy()  # Close the window
            else:
                messagebox.showerror("Error", "Failed to remove student.")
            
            connection.close()  # Close the connection
        else:
            messagebox.showerror("Error", "Failed to connect to the database.")

    # Delete button to trigger the delete function
    tk.Button(operation_window, text="Delete", command=delete).grid(row=1, column=0, columnspan=2, pady=20)

## Operation: Mark Entry

In [60]:
# Operation: Mark Entry
def mark_entry():
    operation_window = tk.Toplevel()
    operation_window.title("Mark Entry")

    # Create Student ID input field
    tk.Label(operation_window, text="Student ID").grid(row=0, column=0, padx=10, pady=5)
    student_id_entry = tk.Entry(operation_window)
    student_id_entry.grid(row=0, column=1, padx=10, pady=5)

    # Fields for marks
    fields = ["Tamil Marks", "English Marks", "Maths Marks", "Science Marks", "Social Marks"]
    entries = {}

    for i, field in enumerate(fields):
        tk.Label(operation_window, text=field).grid(row=i + 1, column=0, padx=10, pady=5)
        entry = tk.Entry(operation_window)
        entry.grid(row=i + 1, column=1, padx=10, pady=5)
        entries[field] = entry

    # Submit function to handle mark entry
    def submit():
        student_id = student_id_entry.get()
        if not student_id:
            messagebox.showwarning("Input Error", "Student ID is required!")
            return

        # Get the marks entered
        marks = [entries[field].get() for field in fields]
        
        # Validate that all marks are numbers between 0 and 100
        if any(not mark.isdigit() or int(mark) < 0 or int(mark) > 100 for mark in marks):
            messagebox.showwarning("Input Error", "Marks must be between 0 and 100!")
            return
        
        # Create database connection
        connection = create_db_connection(host_name, user_name, pass_word, database_name)
        
        if connection:
            # Calculate total marks and grade
            total = sum(map(int, marks))
            grade = "A" if total >= 400 else "B" if total >= 300 else "C"

            # SQL query for inserting or updating marks
            query = """
                INSERT INTO Marks (student_id, tamil_marks, english_marks, maths_marks, science_marks, social_marks, total_marks, grade) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s) 
                ON DUPLICATE KEY UPDATE 
                    tamil_marks=VALUES(tamil_marks), 
                    english_marks=VALUES(english_marks),
                    maths_marks=VALUES(maths_marks),
                    science_marks=VALUES(science_marks),
                    social_marks=VALUES(social_marks),
                    total_marks=VALUES(total_marks), 
                    grade=VALUES(grade)
            """
            
            # Values to insert or update
            values = [student_id] + marks + [total, grade]
            
            # Execute the query
            success = execute_query(connection, query, values)
            
            if success:
                messagebox.showinfo("Success", "Marks entered/updated successfully!")
                operation_window.destroy()  # Close the operation window
            else:
                messagebox.showerror("Error", "Failed to enter/update marks.")
            
            connection.close()  # Close the connection
        else:
            messagebox.showerror("Error", "Failed to connect to the database.")

    # Submit button to trigger the mark entry
    tk.Button(operation_window, text="Submit", command=submit).grid(row=len(fields) + 1, column=0, columnspan=2, pady=20)


## Operation: View Entire Student Details

In [63]:
import tkinter as tk
from tkinter import messagebox

def view_student_details():
    operation_window = tk.Toplevel()
    operation_window.title("View Student Details")

    # Student ID entry
    tk.Label(operation_window, text="Student ID").grid(row=0, column=0, padx=10, pady=5)
    student_id_entry = tk.Entry(operation_window)
    student_id_entry.grid(row=0, column=1, padx=10, pady=5)

    # Labels to display student details (initially empty)
    details_labels = {}
    fields = ["ID", "Name", "DOB", "Gender", "Address", "Phone", "Email", "Total Fee", "Paid Amount", "Status", "Tamil Marks", "English Marks", "Maths Marks", "Science Marks", "Social Marks", "Total Marks", "Grade"]
    for idx, field in enumerate(fields):
        tk.Label(operation_window, text=field).grid(row=idx + 1, column=0, padx=10, pady=5)
        details_labels[field] = tk.Label(operation_window, text="")
        details_labels[field].grid(row=idx + 1, column=1, padx=10, pady=5)

    def fetch():
        student_id = student_id_entry.get()

        if not student_id:
            messagebox.showwarning("Input Error", "Student ID is required!")
            return

        # Check if the student_id is a valid integer (if your IDs are integers)
        try:
            student_id = int(student_id)
        except ValueError:
            messagebox.showwarning("Input Error", "Please enter a valid numeric Student ID!")
            return

        # Query to retrieve student details along with marks and fees
        query = """
            SELECT s.student_id, s.name, s.dob, s.gender, s.address, s.phone, s.email, 
                   f.total_fee, f.paid_amount, f.status, 
                   m.tamil_marks, m.english_marks, m.maths_marks, m.science_marks, m.social_marks, m.total_marks, m.grade 
            FROM Student s
            LEFT JOIN Fees f ON s.student_id = f.student_id
            LEFT JOIN Marks m ON s.student_id = m.student_id
            WHERE s.student_id = %s
        """
        
        # Assuming you have a function to connect to the DB
        connection = create_db_connection(host_name, user_name, pass_word, database_name)
        
        # Fetching the data using execute_query
        rows = read_query(connection, query, (student_id,))

        # Debugging: Print the query results
        # print(f"Query executed: {query}")
        # print(f"Rows fetched: {rows}")

        if rows:
            # Extract the first row of results (since the query is for a specific student)
            student_data = rows[0]

            # Update the labels with student data
            for idx, field in enumerate(fields):
                details_labels[field].config(text=str(student_data[idx]))
        else:
            messagebox.showwarning("No Data", f"No student found with ID {student_id}.")

        # Close the database connection
        connection.close()

    # Button to fetch student details
    tk.Button(operation_window, text="Fetch Details", command=fetch).grid(row=len(fields) + 1, column=0, columnspan=2, pady=10)


## Main Menu Gui

### Main menu functionality

In [67]:
# Main menu functionality
def main_menu():
    # Create main menu window
    menu_window = tk.Tk()
    menu_window.title("Student Information System - Operations")
    
    # Set a fixed window size (optional)
    menu_window.geometry("600x600")
    
    # Create a label for instructions
    tk.Label(menu_window, text="Select Operation", font=("Arial", 16)).pack(pady=20)
    
    # Define a list of operations (buttons)
    operations = [
        ("Add New Student", add_new_student),
        ("Update Student Details", update_student_details),
        ("Remove Student", remove_student),
        ("Mark Entry", mark_entry),
        ("View Entire Student Details", view_student_details)
    ]
    
    # Create a button for each operation
    for op_name, op_command in operations:
        tk.Button(menu_window, text=op_name, command=op_command, width=30, height=2).pack(pady=10)

    # Optional: Add a quit button to close the application
    tk.Button(menu_window, text="Quit", command=menu_window.destroy , width=30, height=2, bg="red").pack(pady=10)

    # Start the Tkinter main loop to display the window
    menu_window.mainloop()

## Login window

In [70]:
# Login window
login_window = tk.Tk()
login_window.title("Student Information System - Login")

login_window.geometry("300x300")

tk.Label(login_window, text="Username").grid(row=0, column=0, padx=10, pady=10)
username_entry = tk.Entry(login_window)
username_entry.grid(row=0, column=1, padx=10, pady=10)

tk.Label(login_window, text="Password").grid(row=1, column=0, padx=10, pady=10)
password_entry = tk.Entry(login_window, show="*")
password_entry.grid(row=1, column=1, padx=10, pady=10)

login_button = tk.Button(login_window, text="Login", command=login)
login_button.grid(row=2, column=0, columnspan=2, pady=20)

login_window.mainloop()

MySQL Database connection successful
MySQL Database connection successful
